# Lab 11 Examples (Huffman Trees)

Click Shift+Enter in each code cell to run the code. Be sure to start with the `#include` directives to load the required libraries.

In [1]:
// For Lab 11, we are limited to using only the following #include directives:

#include <iostream>
#include <string>
#include <iomanip>
#include <cmath>

# Overview

In today's lab, we will look at Huffman Trees. A Huffman Tree is a data structure used for data compression. We will build a Huffman Tree and use it to decode an encoded string. Before we look specifically at Huffman Trees, though, let's review trees and binary trees in general.

## Trees and Binary Trees

A **tree** is a special type of graph that has the following properties:
- It is connected, meaning there is a path between any two nodes.
- It has no cycles, meaning there is no way to start at a node and follow a path that leads back to the same node.
- It has a designated root node, which is the topmost node in the tree.

Outside, a tree's roots are at the bottom, and they branch upward. Trees as a data structure are the opposite: the root is at the top, and the branches extend downward.

A **binary tree** is a type of tree where each node has at most two children, a left child and a right child.

Here is an example of a simple binary tree:

```text
        A
       / \
      B   C
     / \   \
    D   E   F
```

In a **binary search tree**, the left child of a node contains a value less than its parent node, and the right child contains a value greater than its parent node. This makes it easy to find values on average in $\mathcal{O}(log~n)$.

```text
        5
       / \
      3   8
         / \
        7   9
```

## Representing Trees in C++

Unlike with arrays, where elements are stored in contiguous memory locations, trees are typically represented using nodes that contain data and pointers. The node's data field holds the value stored by the node, and the pointers link to other nodes in the tree. In a binary tree, each node will typically have two pointers, one to each child. This allows the tree to be traversed from the root down to the leaves.

But a tree can also have a pointer to its parent node, which allows traversal from child to parent.

The following example is not a Huffman Tree, but it is a simple binary tree that illustrates how nodes can be connected and traversed.

In [2]:
struct Node {
    int data;
    Node* left;
    Node* right;

    Node(int val) : data(val), left(nullptr), right(nullptr) {}
};

Node* root = new Node(5);
Node* current = root;

current->left = new Node(3);
current->right = new Node(8);

current = root;
printf("Current Node Data: %d\n", current->data);

current = current->left;
printf("Current Node Data: %d\n", current->data);

current = root->right;
printf("Current Node Data: %d\n", current->data);

Current Node Data: 5
Current Node Data: 3
Current Node Data: 8


In [3]:
void printSimple(Node* node, int depth = 0) {
    if (node == nullptr)
        return;
    printSimple(node->right, depth + 1);
    for (int i = 0; i < depth; ++i) 
        printf("    " );
    printf("%d\n", node->data);
    printSimple(node->left, depth + 1);
}

printSimple(root);

    8
5
    3


In [4]:
// Let's next add a couple children to node 8 and then print the tree again.
root->right->left = new Node(7);
root->right->right = new Node(9);
printSimple(root);

        9
    8
        7
5
    3


In [7]:
// And now we'll set our current pointer to the root,
// and traverse all the way down the right side of the tree.
current = root;
while (current != nullptr) {
    printf("Current Node Data: %d\n", current->data);
    current = current->right;
}

Current Node Data: 5
Current Node Data: 8
Current Node Data: 9


In [8]:
// Now that we're finished with our simple example tree,
// let's delete our dynamically allocated nodes and set
// our pointers to NULL.

void deleteTree(Node* ptr) {
    if (ptr == nullptr)
        return;

    deleteTree(ptr->left);
    deleteTree(ptr->right);
    delete ptr;
}

deleteTree(root);
root = nullptr;
current = nullptr;

## Huffman Trees

A **Huffman Tree** is a specific type of binary tree used for data compression. It is built based on the frequency of characters in a given dataset. The idea is to assign shorter binary codes to more frequent characters and longer codes to less frequent characters, which helps reduce the overall size of the data when encoded.

The `char` datatype in C++ is associated with a unique ASCII code value. The size of each `char` is typically 1 byte (8 bits), which allows for 256 different characters (from 0 to 255 in decimal).  If the alphabet of the data we want to encode is smaller than 256 characters, we can use fewer number of bits per character.

Link to [ASCII Table](https://www.lookuptables.com/text/ascii-table)

In today's lab, we'll be working with an alphabet of only 6 characters: A, B, C, D, E, and F. With 6 characters, we only need 3 bits to represent each character using a fixed-length encoding scheme:

```text
A: 000
B: 001
C: 010
D: 011
E: 100
F: 101
```

If the data that uses this alphabet is 128 characters long, using a fixed-length encoding scheme such as this one would require $128 * 3 = 384$ bits to encode the message.

Can we do better? Do we need three bits (a fixed length scheme) for every character? What if some characters are more common than others? Could we give those more frequent characters shorter codes?

How about this? Do you see a problem with this encoding scheme?

```text
A: 10
B: 101
C: 100
D: 11
E: 1
F: 110
```

With the fixed-length scheme, we always knew where the end of one character was and the beginning of the next character. Each codeword was exactly 3 bits long.

With a variable-length scheme like this, if we read a 1, how do we know whether we're reading the letter E (codeword 1) or the start of the letter A (codeword 10) or the start of B (codeword 101)? This is the **prefix problem**. We must ensure that no codeword is the first part (or prefix) of another codeword.

We can achieve a variable length encoding scheme without the prefix problem by using the Huffman algorithm.

## The Huffman Algorithm

* Assume that we have an alphabet of 6 characters: `A`, `B`, `C`, `D`, `E`, and `F`. And probabilistically, they occur with the following frequencies:

```text

Character Frequency Percentage
A               .05
B               .09
C               .12
D               .13
E               .16
F               .45
```

* Our goal is to build a tree such that the less frequently occurring characters are closer to the bottom of the tree, and the more frequently occurring characters are closer to the top. So, we start by selecting the two characters with the lowest frequencies, `A` and `B`.  `A` and `B` will be leaf nodes and we will connect them to a newly created parent.

```text
        *    Node AB (.05 + .09 = .14)
       / \
      A   B
```

```text

Character Frequency Percentage
AB              .14
C               .12
D               .13
E               .16
F               .45
```

* We replaced `A` and `B` in our list with our combined node `AB`. 
* Next, we select the two lowest frequency nodes, `C` and `D`, and create a new parent node `CD`.

```text
        *    Node CD (.12 + .13 = .25)
       / \
      C   D
```

* Our updated list is now:

```text
Character Frequency Percentage
AB              .14
CD              .25
E               .16
F               .45
```

* Next, we select the two lowest frequency nodes, `AB` and `E`, and create a new parent node `ABE`.

```text
        *    Node ABE (.14 + .16 = .30)
       / \
      AB  E
     / \
    A   B
```

* Our updated list is now:

```text
Character Frequency Percentage
CD              .25
ABE             .30
F               .45
```

* Next, we select the two lowest frequency nodes, `CD` and `ABE`, and create a new parent node `CDABE`.

```text
          *     Node CDABE (.25 + .30 = .55)
       /     \
     CD      ABE
    / \      / \
   C  D     AB  E
           / \
          A   B 
```

* Our updated list is now:

```text
Character Frequency Percentage
CDABE          .55
F              .45
```

* Next we select the two lowest frequency nodes, `F` and `CDABE`, and create a new parent node `FCDABE`, which will be called the `root`.

```text
              *              Root Node (.45 + .55 = 1.0)
           /      \
          F      CDABE
                /     \
              CD      ABE
             / \      / \
            C  D     AB  E
                    / \
                   A   B 
```

* Note that when connecting nodes with a common parent, the left child is always the node with the lower frequency. This is just a convention that we will follow, so that our approach is consistent, and we all work with the same encoding.

* Now that we have built our Huffman Tree, we can assign binary codes by traversing the tree from the root to each leaf. For each left edge, we append a `0` to the codeword and for each right edge, we append a `1`.

```text
F : 0     to reach leaf F, start at root, traverse left (0)
C : 100   to reach leaf C, start at root, traverse right (1), then left (0), then left (0).
D : 101   to reach leaf D, start at root, traverse right (1), then left (0), then right (1).
A : 1100  to reach leaf A, start at root, traverse right (1), then right (1), then left (0), then left (0).
B : 1101  to reach leaf B, start at root, traverse right (1), then right (1), then left (0), then right (1).
E : 111   to reach leaf E, start at root, traverse right (1), then right (1), then right (1).
```


## Full Huffman Tree

<img src="https://latessa.github.io/cpp-labs/images/Lab11/huffman_tree.png" alt="Full Huffman Tree" style="width:80%">

How would we encode the string `FED`?

Huffman Encoded string: `0111101`

## Compression Ratio

Using our Huffman encoding scheme, we can calculate the average number of bits per character:

```text
Character Frequency Percentage Codeword Length Contribution
A               .05               4            .20
B               .09               4            .36
C               .12               3            .36
D               .13               3            .39
E               .16               3            .48
F               .45               1            .45
-----------------------------------------------
Total Average Bits per Character:               2.24
```

On average, we need only 2.24 bits per character.

To determine the fixed-length bit length we compute:

$$
\text{bits}_{\text{fixed}} \;=\; \big\lceil \log_2(\text{alphabet size}) \big\rceil
\;=\; \big\lceil \log_2 6 \big\rceil \;=\; 3
$$

For an example text of \(n=100\) characters in decoded form:

$$
\begin{aligned}
\text{Fixed-length bits} &= 100 \times 3 = 300\ \text{bits} \\
\text{Huffman bits}      &= 100 \times 2.24 = 224\ \text{bits} \\
\text{Compression ratio} &= \dfrac{224}{300} \approx 0.7467 \;(\approx 74.67\%) \\
\text{Space savings}     &= 1 - 0.7467 \approx 0.2533 \;(\approx 25.33\%)
\end{aligned}
$$

Thus, the Huffman-encoded file is about 74.67% the size of the fixed-length encoded file (≈25.33% smaller).

## Sample Lab 11 Output Using the Frequencies Above

```bash
├── (1)
│   ├── (0.55)
│   │   ├── (0.3)
│   │   │   ├──E (0.16)
│   │   │   └── (0.14)
│   │   │       ├──B (0.09)
│   │   │       └──A (0.05)
│   │   └── (0.25)
│   │       ├──D (0.13)
│   │       └──C (0.12)
│   └──F (0.45)

Encoded string: 0110010011101100100111101000
Decoded string: FACEFACEDFFF
Fixed Length Bits: 36 bits
Average Huffman Bits: 26.88 bits
Compression Ratio (Huffman / Fixed Length): 0.75
Space Savings: 25.33%
```

#### The lab assignment will apply the same 6-character alphabet but with `different` frequencies.

We will now look through the lab assignment and the TODO items in the provided starter code.